In [40]:
# Ch. 10 - Functions
# First-class functions - coined in the 60s by British computer scientist Christopher Strachey - allows for functions
# to be passed into other functions, stored as variables, and returned from other functions.
# In Python: (1) functions can be introspected using the type() function (2) functions can be assigned to variables
# (3) functions can be passed to other functions and returned from functions.
def foo():
    'docstring for foo'
    print('Invoked foo')

print(type(foo))
bar = foo
def get_foo():
    return foo

print(get_foo)
print(get_foo())

# 10.3 Invoking functions
# Functions are callable. Because they are callable, they can be invoked.
print(hasattr(foo, '__call__'))
print('__call__' in dir(foo))
print('='*10)

# On a related note, if a class implements the __call__ method, you can invoke instances of the class.
class CallMe:
    def __call__(self):
        print('called')

c = CallMe()
print('__call__' in dir(c))
c()


<class 'function'>
<function get_foo at 0x00000158CDBD9D00>
<function foo at 0x00000158CDBDBB00>
True
True
True
called


In [41]:
# 10.4 Functions have attributes.
# Calling dir() on the function shows the attributes. They are callable, and have a __call__ method.
# __name__ and __doc__ are for the function name and docstring.
# PEP 232 introduced "Function Attributes", which allow setting and getting arbitrary members to function instances. This
# data is stored in the __dict__ dictionary.
foo.note = 'more info'
print(foo.__dict__)

# Another function attribute is "__defaults__" (same as "func_defaults" in Python 2).
# **It is suggested that default parameters use only non-mutable types.**
# However, Python beginners use [] as default parameter, which leads to errors. To understand why, it is important to
# understand when the default parameters are created - they are initialized during function definition time, which occurs
# at either module load time or during program execution. When the function is created, the default parameters are examined
# and stored in the __defaults__ attribute.

def positive(items, seq=[]):
    for item in items:
        if item > 0:
            seq.append(item)
    return seq

print(positive)
print(positive.__defaults__)
print('='*10)
# On the first invocation of "positive", it appears to behave correctly. The list "seq" is still the same instance defined
# during function creation. It is now populated from the interaction of the previous interaction.
print('Invocation # 1')
print(positive([1]))
print(positive.__defaults__)

print('Invocation # 2')
print(positive([-2]))
print(positive.__defaults__)

print('Invocation # 3')
print(positive([3]))
print(positive.__defaults__)
print('='*10)

# The general solution to resolve the need for mutable default parameters is to shift their creation time from module import time
# to runtime. A common idiom is to have the parameter default to None and then check for that value in the function body.
def positive2(items, seq=None):
    seq = seq or []
    for item in items:
        if item > 0:
            seq.append(item)
    return seq

print('Invocation # 1')
print(positive2([1]))
print(positive2.__defaults__)

print('Invocation # 2')
print(positive2([-2]))
print(positive2.__defaults__)

print('Invocation # 3')
print(positive2([3]))
print(positive2.__defaults__)
print('='*10)

# Or a more pythonic solution would be to use a list comprehension.
items = [3, 4, -5]
pos = [x for x in items if x > 0]
print(pos)


{'note': 'more info'}
<function positive at 0x00000158CDBDA980>
([],)
Invocation # 1
[1]
([1],)
Invocation # 2
[1]
([1],)
Invocation # 3
[1, 3]
([1, 3],)
Invocation # 1
[1]
(None,)
Invocation # 2
[]
(None,)
Invocation # 3
[3]
(None,)
[3, 4]


In [42]:
# 10.5 Function scope
# Functions are defined in a scope. Within the body of a function, you can access the function itself, as well as
# anything that was in scope during the definition of the function. This enables recursion.
# Any variable instances created within a function but not returned from it are local to the function and will be
# garbage collected when the function exits.
import sys
def my_multiply(a, b):
    return a * b

print(my_multiply(2, 10))
try:
    print(i)
except:
    print(f'In exception: {sys.exc_info()}')

# Given that the function name is in global scope within the body of that function, it is possible to attach data
# to the function while the function is executing. It is also possible to attach data to a function outside of
# the function, even before it is ever executed.
def foo3():
    print(foo3.stuff)

foo3.stuff = 'Hello, World!'
foo3()

# The built-in functions "locals" and "globals" return a mapping of names to objects that their respective namespaces contain.
def local_test():
    a = 1
    b = 2
    print(locals())
local_test()
# globals()  # this works, but returns a lot of text!


20
In exception: (<class 'NameError'>, NameError("name 'i' is not defined"), <traceback object at 0x00000158CDFADB40>)
Hello, World!
{'a': 1, 'b': 2}


In [43]:
# 10.6 Functions can be nested - and we can return a function from a function
def adder():
    def add(x, y):
        return x + y
    return add

print(adder())
print(adder()(3, 5))

# If the above example were in a Python program, then the "adder" function would be created at module import time. The
# inner function "add", on the other hand, does not exist during module import. It is created at runtime, when adder
# is invoked. Each invocation of adder creates a new instance of add, as seen by the changing id below:
fn = adder()
print(fn)
fn = adder()
print(fn)
print('='*10)

# Nested function has read/write access to built-ins and globals. Also nested functions have read-only access to
# variables defined in the enclosing functions
x = 5  # "global" variable
y = 3
def wrapper():
    def inner():
        # can't write x unless global is used i.e. UnboundLocalError
        global x
        x = 6
        y = -2  # now local shadows global
        # z is a "free" variable in inner
        print('Inner ', x, y, z)
    y = 1  # now local
    z = 0
    inner()
    print('Wrap', x, y, z)

wrapper()
print('='*10)

# In the example above, x and y are global variables. Within wrapper, read-only access is available for global x.
# Inside of inner function, x is marked with the global keyword, which marks x as the reference to global x.
# In both functions, wrapper and inner, y "shadows" the global y.
# At any point inside a function, when a variable is defined, it becomes local, unless it was previously marked
# with a global keyword.
# A "free" variable - a variable that is neither local nor passed in as an argument to a function.

# Python 3 introduced the nonlocal keyword which allows for finer grained modification of non-global variables.
# The code below throws an error at the line indicated.
#def wrapper():
#    b = 8
#    def inner():
#        print('1 ', b)  # SyntaxError: name 'b' is used prior to nonlocal declaration
#        nonlocal b
#        b = 10
#    print('2 ', b)
#    inner()
#    print('3 ', b)
#wrapper()

def wrapper():
    b = 8
    def inner():
        nonlocal b
        print('2 ', b)
        b = 10
    print('1 ', b)
    inner()
    print('3 ', b)

wrapper()


<function adder.<locals>.add at 0x00000158CDFA18A0>
8
<function adder.<locals>.add at 0x00000158CDFA1440>
<function adder.<locals>.add at 0x00000158CDBDA340>
Inner  6 -2 0
Wrap 6 1 0
1  8
2  8
3  10


In [44]:
# Ch. 11 - Function Parameters
# Python supports four different types of function parameters
# Normal parameter - have a name and position
# Keyword (default/named) parameter - have a name
# Variable parameters - preceded by an asterisk (*), have a position
# Variable keyword parameters - preceded by two asterisks (**), have a name

# Parameters vs arguments: Parameters are the names of variables accepted for input in the function definition.
# Arguments are the variables passed into an invoked function. In the code below, a and b are paramters, x and y are arguments.
def my_mult(a, b):
    return a * b
x = 3
y = 2
print(my_mult(x, y))


6


In [45]:
# 11.2 - Normal and keyword parameters
# Most Python developers run into the mutable default parameter gotcha at some point. The only real difference between
# these two parameter types is that normal parameters are always required in functions declaring them. Like keyword
# parameters, normal parameters also support using the name=value argument style during invocation. Order is not
# important if argument names are provided during invocation.
def normal(a, b, c):
    print(a, b, c)

normal(1, 2, 3)
normal(a=1, b=2, c=3)
normal(c=3, a=1, b=2)

# 11.3 Variable parameters
# Variable parameters allow a function to take arbitrary number of position based arguments. An example of the
# printf function from the C world follows. (Very basic implementation; doesn't support numeric formats.)
def my_printf(fmt, *args):
    done = False
    start = 0
    tup_idx = 0
    while not done:
        i = fmt.find('%s', start)
        if i == -1:
            done = True
        else:
            word = str(args[tup_idx])
            tup_idx += 1
            fmt = fmt[:i] + word + fmt[i+2:]
            start = i + 1 + len(word)
    print(fmt)

print('='*20)
my_printf('Hello')
my_printf('My name: %s', 'Matt')
my_printf('nums: %s, %s, %s', *range(1, 4))
my_printf('nums: %s, %s, %s', 1, 2, 3)
#my_printf('floats: %7.2f, %5.4f, %15.2f', 1.3287283, 34.908347398745, -98734.9823728937288990)
print('floats: %7.2f, %5.4f, %15.2f'%(1.3287283, 34.908347398745, -98734.9823728937288990))

# The common convention is to spell the variable parameter as *args, although the Python interpreter does not care.
# The variable args will be a tuple containing all the arguments passed into the function. We can also pass 0 arguments.
def demo_args(*args):
    print(type(args), args)

print('='*20)
demo_args()
demo_args(1)
demo_args(1,)
demo_args(3, 'foo')

# Variable parameters are commonly combined with variable keyword parameters. Together they can be seen in the construction
# of subclasses. This allows a subclass to easily accept any of the arguments for a parent class without enumerating any
# of them. Variable parameters and variable keyword parameters are also used for decorators.


1 2 3
1 2 3
1 2 3
Hello
My name: Matt
nums: 1, 2, 3
nums: 1, 2, 3
floats:    1.33, 34.9083,       -98734.98
<class 'tuple'> ()
<class 'tuple'> (1,)
<class 'tuple'> (1,)
<class 'tuple'> (3, 'foo')


In [46]:
# 11.4 - The * operator
# The asterisk serves to:
#   1) apply multiplication (4*2)
#   2) apply the power operator (4**2)
#   3) mark variable parameters
#   4) flatten argument sequences, the splat operator
#   5) mark variable keyword parameters
#   6) flatten keyword dictionaries, the splat operator
vars = ['John', 'Paul']
demo_args(*vars)

# If the * is left off the vars argument, then args would be a tuple containing a single item - the list with two elements.
# This is probably not what was intended if vars was meant to contain the parameters for a function.
demo_args(vars)

# In Python 2, only a single sequence could be flattened into a function. In Python 3, it is allowed.
# The invocation below fails in Python 2; see the execution in Python 2 below.
demo_args(*vars, *vars)

# ======================================================
# $ python
# Python 2.7.12 (default, Oct 10 2016, 12:56:26)
# [GCC 5.4.0] on cygwin
# Type "help", "copyright", "credits" or "license" for more information.
# >>>
# >>> def demo_args(*args):
# ...     print(type(args), args)
# ...
# >>>
# >>> vars = ['John', 'Paul']
# >>>
# >>> demo_args(*vars)
# (<type 'tuple'>, ('John', 'Paul'))
# >>>
# >>> demo_args(*vars, *vars)
#   File "<stdin>", line 1
#     demo_args(*vars, *vars)
#                      ^
# SyntaxError: invalid syntax
# >>>
# >>>
# ======================================================

# If a function has normal, keyword and variable paramters, it may be invoked with just a flattened sequence.
# In that case, the sequence will populate the normal and keyword parameters, and any left over variables will
# be left in the variable argument.
print('='*20)
def func(a, b='b', *args):
    print([x for x in [a, b, args]])

vars = (3, 4, 5)
func(*vars)

# Since * flattens the arguments, they fill out the parameters. The above invocation is the same as calling
# the function with the arguments listed out.
func(vars[0], vars[1], vars[2])


<class 'tuple'> ('John', 'Paul')
<class 'tuple'> (['John', 'Paul'],)
<class 'tuple'> ('John', 'Paul', 'John', 'Paul')
[3, 4, (5,)]
[3, 4, (5,)]


In [47]:
# 11.5 Variable keyword parameters
import sys
def demo_kwargs(**kwargs):
    print(type(kwargs), kwargs)

demo_kwargs()
demo_kwargs(one=1)
demo_kwargs(one=1, two=2)

# The error below can be confusing, since demo_kwargs takes zero *normal* parameters, but any number
# of keyword parameters.
try:
    demo_kwargs(1)
except:
    print(f'Error: {sys.exc_info()}')

# 11.6 - Flattening dictionaries
# The double asterisks also serves to flatten - or splat - a dicitonary into keyword arguments for a function.
def distance(x1, y1, x2, y2):
    return ((x1-x2)**2 + (y1-y2)**2) ** 0.5

points = {'x1':1, 'y1':1, 'x2':4, 'y2':5}
print(distance(**points))

# The above invocation is the same as calling the function with the dictionary items listed as keyword arguments.
print(distance(x1=1, y1=1, x2=4, y2=5))
demo_kwargs(**points)


<class 'dict'> {}
<class 'dict'> {'one': 1}
<class 'dict'> {'one': 1, 'two': 2}
Error: (<class 'TypeError'>, TypeError('demo_kwargs() takes 0 positional arguments but 1 was given'), <traceback object at 0x00000158CDBEA080>)
5.0
5.0
<class 'dict'> {'x1': 1, 'y1': 1, 'x2': 4, 'y2': 5}


In [48]:
# 11.7 - Arbitrary function parameters
# A function that has both variable parameters and variable keyword parameters can take an arbitrary number of arguments.
# This makes using the combination of them prime candidates for the parameters of subclass constructors and decorators.
# Their order in the function definition should be as: normal, keyword, variable, variable keyword
def demo_params(normal, kw="Test", *args, **kwargs):
    print(normal, kw, args, kwargs)

args = (0, 1, 2)
kw = {'foo': 3, 'bar': 4}
demo_params(*args, **kw)

# The above invocation is equivalent to the following:
demo_params(args[0], args[1], args[2], foo=3, bar=4)


0 1 (2,) {'foo': 3, 'bar': 4}
0 1 (2,) {'foo': 3, 'bar': 4}


In [49]:
# Ch. 12 - Closures
# A closure in Python is simply a function that is returned by another function.
def add_x(x):
    def adder(num):
        # adder is a closure; x is a free variable
        return x + num
    return adder

add_5 = add_x(5)
print(add_5)
print(add_5(10))

# In Python, functions can return new functions. The inner function is a closure and any variable
# it accesses that are defined outside of that function are free variables.

# 12.1 - Common uses of closures
#   1) To keep a common interface (the adapter pattern)
#   2) To eliminate code duplication
#   3) To delay execution of a function


<function add_x.<locals>.adder at 0x00000158CDBDB1A0>
15


In [50]:
# Ch. 13 - Decorators
# A decorator is a design pattern that allows behavior to be added to an existing object dynamically -- Wikipedia.
# In Python, a decorator is a method for altering a callable (functions or methods). Closures enable the creation
# of decorators. A decorated callable can be altered at the following times: 1) before 2) during 3) after invocation.
def verbose(func):
    def wrapper():
        print(f'Before {func.__name__}')
        result = func()
        print(f'After {func.__name__}')
        return result
    return wrapper

# A decorator is really only userful when applied to a function. There are two ways to do this.
# Method 1) Simply invoke the decorator on a function.
def hello():
    print('Hello')
hello = verbose(hello)
print(hello.__name__)   # The new function is actually "wrapper"
print('='*20)
hello()


wrapper
Before hello
Hello
After hello


In [51]:
# As described in PEP 318, Python 2.4 provided a second method of wrapping a function. It is shown below.
# Note that there are no parentheses after the decorator name.
@verbose
def greet():
    print("G'day")
greet()


Before greet
G'day
After greet


In [52]:
# A closure that takes parameters.
def chatty(func):
    def wrapper(*args, **kwargs):
        print(f'Before {func.__name__}')
        result = func(*args, **kwargs)
        print(f'After  {func.__name__}')
        return result
    return wrapper

@chatty
def mult(x, y):
    return x * y

print(mult(2, 4))
print('='*30)

@chatty
def goodbye():
    print('Later')

goodbye()


Before mult
After  mult
8
Before goodbye
Later
After  goodbye


In [53]:
# 13.2 A decorator template
def decorator(func_to_decorate):
    def wrapper(*args, **kwargs):
        # do something before invocation
        result = func_to_decorate(*args, **kwargs)
        # do something after
        return result
    wrapper.__doc__ = func_to_decorate.__doc__
    wrapper.__name__ = func_to_decorate.__name__
    return wrapper

# In well-behaved decorators the __doc__ and __name__ of the wrapper function need to be updated with the values
# from the function that is being decorated. For "pickling" (serialization to disk) of objects, it is required
# that __name__ is updated. The __doc__ attribute is updated so that the function is friendly to introspection.

# functools.wraps, which is a decorator itself, will update __doc__ and __name__ as well.
# It will also update __module__ attribute. So another template is:
import functools
def decorator(func_to_decorate):
    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        # do something before invocation
        result = func_to_decorate(*args, **kwargs)
        # do something after
        return result
    return wrapper

# Any callable can wrap another function. So a callable can also serve as a decorator. Here is an example
# of a class that can be used to decorate functions.
class decorator_class(object):
    def __init__(self, function):
        self.function = function
    def __call__(self, *args, **kwargs):
        # do something before invocation
        result = self.function(*args, **kwargs)
        # do something after
        return result

# This class decorator can be used as follows:
'''
@decorator_class
def function():
    # implementation
'''


'\n@decorator_class\ndef function():\n    # implementation\n'

In [54]:
# 13.3 Parameterized decorators
# Often a decorator needs to be customized on a per function basis. This can be achieved by parameterized decorators.
# E.g. Django @require_http_methods(). Invocations: @require_http_methods(['GET','POST'])
# The usual manner is - use a closure to generate a new function. Hence the method to generate a parameterizable
# decorator is to wrap the decorator with another function! The parameterized decorator itself is actually
# a decorator generator.
# ==================================
# Let's say we want to truncate results from functions to length 5. A single decorator could do that.
def trunc(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result[:5]
    return wrapper

@trunc
def data():
    return 'foobar'

print(data())

# Now assume the requirement for truncation has changed. One function must be truncated to length of 3 while
# another function might need to have a length of 6. What follows is a simple parameterized decorator - or a
# function taht generates customized decorators - that limits the length of the result of the decorated function.
def limit(length):
    def decorator(func):
        def wrapper(*args, **kwargs):
            result = func(*args, **kwargs)
            return result[:length]
        return wrapper
    return decorator

@limit(3)
def data3():
    return 'limit to 3'

print(data3())

@limit(6)
def data6():
    return 'limit to 6'

print(data6())

# The syntactic sugar for decorating with parameterized decorators is to declare the decorator before the
# function definition.
@limit(3)
def data3():
    return 'limit to 3'

# It is the same as the following:
def data3():
    return 'limit to 3'
data3 = limit(3)(data3)

# When the function limit is invoked (with 3 as its parameter), it returns (or generates) a function - a decorator.
# This decorator, aptly named "decorator", is then invoked with the function to decorate, data3.


fooba
lim
limit 


In [55]:
# 13.4 Parameterized template
def param_dec(option):
    def decorator(function):
        def wrapper(*args, **kwargs):
            # probably use option in here
            # before
            result = function(*args, **kwargs)
            # after
            return result
        wrapper.__doc__ = function.__doc__
        wrapper.__name__ = function.__name__
        return wrapper
    return decorator


In [56]:
# 13.5 Multiple decorators
# Just as functions can be nested arbitrarily inside of other functions, functions can be wrapped by multiple decorators.
@chatty
@limit(2)
def greet():
    return 'Greetings'

print(greet())
print('='*30)

# The decorating syntactic sugar is the same as:
greet = chatty(limit(2)(greet))
greet()


Before wrapper
After  wrapper
Gr
Before wrapper
Before wrapper
After  wrapper
After  wrapper


'Gr'

In [57]:
# 13.6 Common uses for decorators
# Decorators can alter or inspect the:
# -  function arguments
# -  function being wrapped
# -  results of the function
# Common instances where decorators are used:
# -  caching expensive calculations
# -  retrying a function that might fail
# -  redirecting sys.stdout to capture what a function prints to it
# -  logging the amount of time spent in a function
# -  timing out a function call
# -  access control


In [58]:
# Ch. 14 - Alternate decorator implementations
# Skipped for now
